# Performance Comparison of Sparse Reward Models with LunarLander-v2

This notebook implements a performance comparison experiment for three models in a sparse reward environment: 
- Full Sequence Gradient Planner (エピソードの最初に1回だけ計画)
- Proximal Policy Optimization (PPO)
- Soft Actor-Critic (SAC)

~~Standard Gradient Planner (1-step MPC方式、毎ステップ再計画)~~ は実行時間短縮のため実行しません。

The experiments will be conducted in the `LunarLander-v2` environment wrapped with `SparseRewardWrapper`. We will collect data, train each model, and evaluate their performance.

すべての実験パラメータは一つのセルで管理されており、簡単に変更できるようになっています。

In [ ]:
# Import necessary libraries
import sys
import os

# プロジェクトのルートディレクトリをPythonのパスに追加します
# これにより、'src'ディレクトリからモジュールをインポートできるようになります
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gymnasium as gym
import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
from tqdm.notebook import tqdm  # Jupyterノートブック用のtqdmを使用
from src.utils.environment_wrappers import SparseRewardWrapper
from src.agents.ppo_agent import PPOAgent
from src.agents.sac_agent import SACAgent
from src.agents.planner_agent import SparseRewardPlannerAgent
# 修正した勾配プランナーをインポート
from src.agents.gradient_planner_agent import GradientPlannerAgent
from src.models.world_models import DynamicsModel, TerminalRewardModel

In [ ]:
# Set up the environment
ENV_ID = 'LunarLander-v2'  # Pendulum-v1からLunarLander-v2に変更
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
env = SparseRewardWrapper(gym.make(ENV_ID))

# --- ハイパーパラメータ設定 (LunarLander-v2用に調整) ---

# 一般設定
RANDOM_SEED = 42               # 乱数シード
MODEL_FREE_STEPS = 1000000     # モデルフリー手法の学習ステップ数 (LunarLanderはより複雑なので増加)

# データ収集パラメータ
INITIAL_RANDOM_STEPS = 100000  # 初期データ収集ステップ数 (より多くのデータが必要)

# 評価設定
EVAL_EPISODES = 10             # 評価時のエピソード数

# 勾配プランナーのパラメータ
GRAD_PLAN_HORIZON = 400        # 計画ホライゾン (LunarLanderのエピソードは長いため増加)
GRAD_NUM_ITERATIONS = 2000     # 勾配法の最適化イテレーション数 (より複雑なため増加)
GRAD_LEARNING_RATE = 0.005     # 勾配法の学習率 (より安定した学習のため減少)

# 世界モデル学習パラメータ
WORLD_MODEL_ITERATIONS = 5     # 世界モデル学習の反復回数 (より複雑なため増加)
WORLD_MODEL_STEPS_PER_ITER = 5000  # 各反復で収集するステップ数
WORLD_MODEL_EPOCHS_PER_ITER = 100  # 各反復でのエポック数
WORLD_MODEL_BATCH_SIZE = 1024      # 学習時のバッチサイズ (より大きなバッチで安定化)
WORLD_MODEL_LR = 5e-4          # 世界モデルの学習率 (より安定した学習のため減少)
NEW_DATA_RATIO = 0.8           # 新しく収集したデータの利用率 (0.8=80%使用、古いデータも一部保持)

# モデルフリー手法の共通パラメータ
EVAL_INTERVAL = 10000          # 評価間隔（ステップ）(より頻繁に評価)

# 再現性確保のためのシード設定
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

## Data Collection
In this section, we will collect trajectories for each model to be used for training.

In [ ]:
# Function to collect random trajectories
def collect_random_trajectories(env, num_steps):
    """指定されたステップ数だけランダムに行動してデータを収集する"""
    trajectories = []
    episode_count = 0
    
    # 1エピソード分のデータを一時的に保持するリスト
    states, actions, rewards, next_states = [], [], [], []
    
    state, _ = env.reset()
    total_steps = 0
    
    # tqdmで進捗を表示
    pbar = tqdm(total=num_steps, desc="Collecting random data", leave=False)
    while total_steps < num_steps:
        action = env.action_space.sample()  # ランダムな行動をサンプリング
        next_state, reward, terminated, truncated, _ = env.step(action)
        
        states.append(state)
        actions.append(action)
        rewards.append([reward]) # 報酬の次元を合わせる
        next_states.append(next_state)
        
        state = next_state
        total_steps += 1
        
        # 進捗バーを更新
        pbar.update(1)
        
        done = terminated or truncated
        if done:
            episode_count += 1
            
            # 1エピソードが終了したら、データを辞書形式にまとめてtrajectoriesに追加
            episode_data = {
                'states': np.array(states, dtype=np.float32),
                'actions': np.array(actions, dtype=np.float32),
                'rewards': np.array(rewards, dtype=np.float32),
                'next_states': np.array(next_states, dtype=np.float32)
            }
            trajectories.append(episode_data)
            
            # 次のエピソードのためにリセット
            states, actions, rewards, next_states = [], [], [], []
            state, _ = env.reset()
    
    pbar.close()
    return trajectories

### Collecting Data for Each Model
We will create instances of each agent and collect data.

In [ ]:
# Initialize agents
# デバイス設定を明示的に渡す
# LunarLander-v2は離散行動空間を持っているため、行動空間の処理を調整する必要がある
action_dim = env.action_space.n  # 離散行動空間のサイズ (LunarLander-v2では4)
state_dim = env.observation_space.shape[0]  # 状態空間の次元 (LunarLander-v2では8)

# 連続値エージェント用のダミーaction_high ([-1, 1]の範囲に正規化)
# 注：実際にはLunarLander-v2は離散行動空間のため、この値は内部で適切に変換される必要があります
action_high = np.ones(action_dim)

# PPOとSACエージェントを初期化
ppo_agent = PPOAgent(
    state_shape=env.observation_space.shape, 
    action_shape=(action_dim,),  # 離散行動空間を連続値として扱う
    action_high=action_high, 
    device=device
)
sac_agent = SACAgent(
    state_shape=env.observation_space.shape, 
    action_shape=(action_dim,),  # 離散行動空間を連続値として扱う
    action_high=action_high, 
    device=device
)

# --- 行動系列全体を使用する勾配プランナー (エピソードの最初に1回だけ計画) ---
# まずCPU上でモデルを初期化
full_dynamics_model = DynamicsModel(state_dim=state_dim, action_dim=action_dim)
full_reward_model = TerminalRewardModel(state_dim=state_dim, action_dim=action_dim)
# 学習の直前にGPUに転送
full_dynamics_model = full_dynamics_model.to(device)
full_reward_model = full_reward_model.to(device)
# フル行動系列を使用する勾配プランナーを設定
full_grad_planner = GradientPlannerAgent(
    full_dynamics_model, full_reward_model, 
    action_dim=action_dim,
    action_high=action_high,
    plan_horizon=GRAD_PLAN_HORIZON,
    num_iterations=GRAD_NUM_ITERATIONS,
    learning_rate=GRAD_LEARNING_RATE,
    use_full_sequence=True  # エピソードの最初に1回だけ計画し、行動系列全体を保存
)

# 注意: LunarLander-v2は離散行動空間なので、連続値エージェントを使用する場合は
# 実際に環境に送る前に行動を離散化する必要があります

In [ ]:
# ActionDiscretizerWrapper - 連続値アクション→離散値アクション変換のためのラッパークラス
class ActionDiscretizerWrapper(gym.ActionWrapper):
    def __init__(self, env, action_dim):
        super().__init__(env)
        self.action_dim = action_dim
        
    def action(self, action):
        """連続値の行動を離散値インデックスに変換"""
        if isinstance(action, np.ndarray) and len(action.shape) > 0:
            # 連続値ベクトルから最大値のインデックスを選択
            # 例: [0.1, 0.7, 0.2, 0.0] → 1 (最大値0.7のインデックス)
            return np.argmax(action)
        return int(action)  # 単一の値の場合、整数に変換

# 環境をラップしてアクション変換を可能に
env = ActionDiscretizerWrapper(env, action_dim)

# Collect trajectories for the planner model using random actions
# 最初のシードデータとしてINITIAL_RANDOM_STEPSステップのデータを収集
print(f"ランダム行動で{INITIAL_RANDOM_STEPS}ステップの初期データを収集中...")
planner_trajectories = collect_random_trajectories(env, INITIAL_RANDOM_STEPS) 

# Save the collected data
# Note: You might need to create the 'data' directory first.
import os
os.makedirs('../data', exist_ok=True)
with open('../data/random_trajectories_lunarlander.pkl', 'wb') as f:
    pickle.dump(planner_trajectories, f)

print(f"\n{len(planner_trajectories)} episodes of random data saved.")

## Training the Models
Now we will train each model according to the experiment plan.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# --- Helper function to collect data using the planner ---
def collect_planner_trajectories(env, agent, num_steps):
    """指定されたステップ数だけプランナーを使ってデータを収集する"""
    trajectories = []
    episode_count = 0
    
    states, actions, rewards, next_states = [], [], [], []
    
    state, _ = env.reset()
    total_steps = 0
    
    # use_full_sequence モードを使用している場合はエピソードの始めにリセット
    if hasattr(agent, 'reset'):
        agent.reset()  # エピソードの始めであることを通知
    
    # tqdmで進捗を表示
    pbar = tqdm(total=num_steps, desc="Collecting data", leave=False)
    while total_steps < num_steps:
        # プランナーが行動を計画
        action = agent.exploit(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        
        states.append(state)
        actions.append(action)
        rewards.append([reward])
        next_states.append(next_state)
        
        state = next_state
        total_steps += 1
        
        # 進捗バーを更新
        pbar.update(1)
        
        done = terminated or truncated
        if done:
            episode_count += 1
            episode_data = {
                'states': np.array(states, dtype=np.float32),
                'actions': np.array(actions, dtype=np.float32),
                'rewards': np.array(rewards, dtype=np.float32),
                'next_states': np.array(next_states, dtype=np.float32)
            }
            trajectories.append(episode_data)
            
            # 新しいエピソードの準備
            states, actions, rewards, next_states = [], [], [], []
            state, _ = env.reset()
            
            # use_full_sequence モードを使用している場合は新しいエピソードの始めにリセット
            if hasattr(agent, 'reset'):
                agent.reset()
    
    pbar.close()
    return trajectories

# --- 1. Planner (World Models) Iterative Training ---
def train_world_models_iteratively(planner_agent, env, initial_trajectories, num_iterations=10, steps_per_iteration=1000, epochs_per_iteration=1, batch_size=64, learning_rate=1e-3, initial_random_steps=INITIAL_RANDOM_STEPS, new_data_ratio=1.0):
    # print削除、tqdmでの進捗表示に一元化
    dynamics_model = planner_agent.dynamics_model
    reward_model = planner_agent.reward_model
    
    # オプティマイザ
    dynamics_optimizer = torch.optim.Adam(dynamics_model.parameters(), lr=learning_rate)
    reward_optimizer = torch.optim.Adam(reward_model.parameters(), lr=learning_rate)
    
    # 集約データセット
    aggregated_trajectories = initial_trajectories
    
    # 学習曲線追跡用の辞書
    learning_curve = {}
    
    # 初期のランダムデータ収集ステップ数を反映
    total_steps = initial_random_steps
    
    # 初期評価を省略し、直接学習を開始
    
    # tqdmでイテレーションの進捗を表示
    for iteration in tqdm(range(num_iterations), desc="World Model Training", leave=False):
        # --- Step 1: データセットの準備 ---
        all_states, all_actions, all_rewards_data, all_next_states = [], [], [], []
        for episode in aggregated_trajectories:
            final_reward = np.sum(episode['rewards'])
            all_states.append(torch.tensor(episode['states'], dtype=torch.float))
            all_actions.append(torch.tensor(episode['actions'], dtype=torch.float))
            all_next_states.append(torch.tensor(episode['next_states'], dtype=torch.float))
            all_rewards_data.append({
                'states': torch.tensor(episode['states'], dtype=torch.float).unsqueeze(0),
                'actions': torch.tensor(episode['actions'], dtype=torch.float).unsqueeze(0),
                'terminal_reward': torch.tensor([final_reward], dtype=torch.float)
            })

        # CPU上でテンソルを結合する（GPUへの転送は行わない）
        states_tensor = torch.cat(all_states, dim=0)
        actions_tensor = torch.cat(all_actions, dim=0)
        next_states_tensor = torch.cat(all_next_states, dim=0)
        
        # CPU上のテンソルを使用してデータセットを作成
        dynamics_dataset = TensorDataset(states_tensor, actions_tensor, next_states_tensor)
        dynamics_loader = DataLoader(dynamics_dataset, 
                                     batch_size=batch_size, 
                                     num_workers=6,
                                     pin_memory=True,
                                     shuffle=True)
        
        # 報酬モデル用のデータを準備（エポックのループの外で1回だけ）
        states_batch = []
        actions_batch = []
        rewards_batch = []
        
        # データをバッチ処理用に準備
        for reward_data in all_rewards_data:
            states_batch.append(reward_data['states'])
            actions_batch.append(reward_data['actions'])
            rewards_batch.append(reward_data['terminal_reward'])
            
        # テンソルに変換
        states_batch = torch.cat(states_batch, dim=0)
        actions_batch = torch.cat(actions_batch, dim=0)
        rewards_batch = torch.cat(rewards_batch, dim=0)
        
        # DataLoaderの作成（エポックのループの外で1回だけ）
        reward_dataset = TensorDataset(states_batch, actions_batch, rewards_batch)
        reward_loader = DataLoader(reward_dataset, 
                                  batch_size=min(32, len(reward_dataset)), 
                                  num_workers=6,
                                  shuffle=True,
                                  pin_memory=True)
        
        # --- Step 2: モデルの学習 ---
        epoch_iterator = tqdm(range(epochs_per_iteration), desc=f"Training Epochs", leave=False)
        for epoch in epoch_iterator:
            # DynamicsModelの学習
            dynamics_model.train()
            total_dyn_loss = 0
            progress_bar = tqdm(dynamics_loader, desc=f"Epoch {epoch+1}/{epochs_per_iteration}", leave=False)
            for s, a, next_s in progress_bar:
                # データをここでGPUに転送
                s = s.to(device)
                a = a.to(device)
                next_s = next_s.to(device)
                
                s_in, a_in = s.unsqueeze(1), a.unsqueeze(1)
                pred_next_s = dynamics_model(s_in, a_in).squeeze(1)
                loss = torch.nn.functional.mse_loss(pred_next_s, next_s)
                dynamics_optimizer.zero_grad()
                loss.backward()
                dynamics_optimizer.step()
                total_dyn_loss += loss.item()
            
            # TerminalRewardModelの学習
            reward_model.train()
            total_rew_loss = 0
            
            # バッチ処理
            reward_progress_bar = tqdm(reward_loader, desc=f"Reward Model", leave=False)
            for s_seq, a_seq, target_rew in reward_progress_bar:
                # データをGPUに転送
                s_seq = s_seq.to(device)
                a_seq = a_seq.to(device)
                target_rew = target_rew.to(device)
                
                pred_rew = reward_model(s_seq, a_seq)
                loss = torch.nn.functional.mse_loss(pred_rew.squeeze(), target_rew.squeeze())
                reward_optimizer.zero_grad()
                loss.backward()
                reward_optimizer.step()
                total_rew_loss += loss.item() * s_seq.size(0)  # バッチサイズを考慮
                
        # 各エポックの最後にtqdmを更新
        epoch_iterator.update(1)

        # --- Step 3: 学習済みモデルで新たなデータを収集・集約 ---
        if iteration < num_iterations - 1: # 最後のイテレーションではデータ収集は不要
            # tqdmはcollect_planner_trajectories内で処理
            new_trajectories = collect_planner_trajectories(env, planner_agent, steps_per_iteration)
            
            # 新しいデータ利用率に基づいてデータを集約
            if new_data_ratio >= 1.0:
                # 新しいデータを全て追加
                aggregated_trajectories.extend(new_trajectories)
            elif new_data_ratio <= 0:
                # 新しいデータを使用しない（既存のデータのみ使用）
                pass
            else:
                # 新しいデータをサンプリングして追加
                num_new_to_use = int(len(new_trajectories) * new_data_ratio)
                selected_new = random.sample(new_trajectories, num_new_to_use)
                aggregated_trajectories.extend(selected_new)
            
            # この時点でのモデルの評価を記録
            total_steps += steps_per_iteration
            current_reward = evaluate_model(planner_agent, env, EVAL_EPISODES)
            learning_curve[total_steps] = current_reward
    
    # 最終評価
    final_reward = evaluate_model(planner_agent, env, EVAL_EPISODES)
    if total_steps in learning_curve:
        learning_curve[total_steps] = final_reward
    else:
        learning_curve[total_steps + steps_per_iteration] = final_reward
    
    return learning_curve

# --- Model-Free (PPO/SAC) Training ---
def train_model_free_agent(agent, env, total_steps, eval_interval=5000):
    learning_curve = {}
    state, _ = env.reset()
    
    # tqdmで進捗を表示
    progress_bar = tqdm(range(total_steps), desc=f"Training {agent.__class__.__name__}", leave=False)
    for step in progress_bar:
        log_pi = 0.0
        if isinstance(agent, PPOAgent):
            action, log_pi = agent.explore(state)
        else:
            action = agent.explore(state)
            
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        
        agent.buffer.append(state, action, reward, done, log_pi, next_state)

        if isinstance(agent, PPOAgent):
            if agent.buffer.is_full():
                agent.update()
        elif isinstance(agent, SACAgent):
            agent.update()

        if done:
            state, _ = env.reset()
        else:
            state = next_state
            
        if (step + 1) % eval_interval == 0:
            avg_reward = evaluate_model(agent, env, EVAL_EPISODES)
            learning_curve[step + 1] = avg_reward
            
    return learning_curve

# --- Evaluation Function ---
def evaluate_model(agent, env, num_episodes=10):
    total_reward = 0
    episode_rewards = []
    
    # tqdmを使用してエピソードの進捗を表示
    with tqdm(range(num_episodes), desc="Evaluating", leave=False) as pbar:
        for episode in pbar:
            state, _ = env.reset()
            done = False
            episode_reward = 0
            
            # エージェントがresetメソッドを持っていれば呼び出し
            if hasattr(agent, 'reset'):
                agent.reset()
            
            # エピソード実行
            while not done:
                action = agent.exploit(state)
                state, reward, terminated, truncated, _ = env.step(action)
                episode_reward += reward
                done = terminated or truncated
            
            total_reward += episode_reward
            episode_rewards.append(episode_reward)
            
            # 進捗バーを更新
            pbar.update(1)
    
    avg_reward = total_reward / num_episodes
    return avg_reward

In [ ]:
# --- Execute Training and Evaluation ---

# 2. Train and evaluate the Full Sequence Gradient Planner (1回だけ計画)
# Load pre-collected random data
with open('../data/random_trajectories_lunarlander.pkl', 'rb') as f:
    initial_full_trajectories = pickle.load(f)
    
# 学習曲線データを取得するように変更
full_grad_learning_curve = train_world_models_iteratively(
    planner_agent=full_grad_planner,
    env=env,
    initial_trajectories=initial_full_trajectories,
    num_iterations=WORLD_MODEL_ITERATIONS,
    steps_per_iteration=WORLD_MODEL_STEPS_PER_ITER,
    epochs_per_iteration=WORLD_MODEL_EPOCHS_PER_ITER,
    batch_size=WORLD_MODEL_BATCH_SIZE,
    learning_rate=WORLD_MODEL_LR,
    initial_random_steps=INITIAL_RANDOM_STEPS,
    new_data_ratio=NEW_DATA_RATIO
)
# 最終評価報酬を取得（プロット用）
full_grad_reward = list(full_grad_learning_curve.values())[-1]

# 3. Train and evaluate SAC
sac_agent_train = SACAgent(
    state_shape=env.observation_space.shape, 
    action_shape=(action_dim,),
    action_high=action_high, 
    device=device
)
sac_learning_curve = train_model_free_agent(sac_agent_train, env, MODEL_FREE_STEPS, eval_interval=EVAL_INTERVAL)

# 4. Train and evaluate PPO
ppo_learning_curve = train_model_free_agent(ppo_agent, env, MODEL_FREE_STEPS, eval_interval=EVAL_INTERVAL)

## Results
Finally, we will plot the results of the evaluation.

In [ ]:
# Plotting the results
plt.figure(figsize=(12, 7))

# Plot PPO learning curve
if 'ppo_learning_curve' in locals() and ppo_learning_curve:
    ppo_steps = list(ppo_learning_curve.keys())
    ppo_rewards = list(ppo_learning_curve.values())
    plt.plot(ppo_steps, ppo_rewards, label='PPO', marker='o')

# Plot SAC learning curve
if 'sac_learning_curve' in locals() and sac_learning_curve:
    sac_steps = list(sac_learning_curve.keys())
    sac_rewards = list(sac_learning_curve.values())
    plt.plot(sac_steps, sac_rewards, label='SAC', marker='s')

# Plot Full Sequence Gradient Planner's learning curve as a step function
if 'full_grad_learning_curve' in locals() and full_grad_learning_curve:
    # キーが昇順にソートされていることを確認
    full_steps = sorted(list(full_grad_learning_curve.keys()))
    full_rewards = [full_grad_learning_curve[step] for step in full_steps]
    
    # 値がある場合のみグラフを描画
    if full_rewards:
        # 階段状のグラフを描画
        plt.step(full_steps, full_rewards, where='post', color='r', linestyle='-', 
                marker='+', markersize=8, label=f'Full Seq Gradient Planner (1回計画, Final: {full_rewards[-1]:.2f})')
        
        # 各ポイントをプロット
        plt.scatter(full_steps, full_rewards, color='r', s=50)

plt.xlabel('Total Environment Steps')
plt.ylabel('Average Evaluation Reward')
plt.title('LunarLander-v2における手法の比較')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 各モデルの最適方策を使って動画を記録・表示する
from gymnasium.wrappers import RecordVideo
import matplotlib.pyplot as plt
from IPython.display import HTML
import glob
import os
from IPython import display
import base64

# 動画を表示するヘルパー関数
def show_video(video_path):
    """
    指定したパスの動画をノートブック内に表示する
    """
    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{base64.b64encode(video_file).decode()}"
    return HTML(f"""<video width=600 controls>
                    <source src="{video_url}" type="video/mp4">
                    </video>""")

# 動画を記録するためのディレクトリを作成
video_dir = "../videos/lunarlander"
os.makedirs(video_dir, exist_ok=True)

# 各モデルについて最終的な方策を実行して動画を記録
models = {
    "Full_Sequence_Planner": full_grad_planner,
    "PPO": ppo_agent,
    "SAC": sac_agent_train
}

# 各モデルの評価と動画記録
for model_name, model in models.items():
    print(f"\n--- {model_name} の動画記録 ---")
    
    # 動画記録用のラッパー環境を作成 (render_mode="rgb_array"を指定)
    video_env = RecordVideo(
        ActionDiscretizerWrapper(
            SparseRewardWrapper(gym.make(ENV_ID, render_mode="rgb_array")),
            action_dim
        ),
        video_folder=f"{video_dir}/{model_name}",
        name_prefix=f"{model_name}_episode",
        episode_trigger=lambda x: True  # すべてのエピソードを記録
    )
    
    # エピソード実行
    state, _ = video_env.reset()
    done = False
    total_reward = 0
    
    # エージェントがresetメソッドを持っていれば呼び出し
    if hasattr(model, 'reset'):
        model.reset()
    
    # エピソード実行
    while not done:
        action = model.exploit(state)  # 最適方策での行動
        state, reward, terminated, truncated, _ = video_env.step(action)
        total_reward += reward
        done = terminated or truncated
    
    video_env.close()
    
    print(f"{model_name} の総報酬: {total_reward:.2f}")
    
    # 最新の動画ファイルを取得して表示
    video_files = sorted(glob.glob(f"{video_dir}/{model_name}/*.mp4"))
    if video_files:
        latest_video = video_files[-1]
        print(f"動画ファイル: {latest_video}")
        display.display(show_video(latest_video))
    else:
        print(f"警告: {model_name} の動画ファイルが見つかりませんでした。")